In [ ]:
import os
import yaml
import civis
import requests
import json
import pandas as pd
import urllib.parse
from pathlib import Path
from datetime import date
from datetime import timedelta

# headers needed for GET requests & GLOBALS
GET_HEADERS = {'X-Knack-REST-API-KEY':'1a210580-315e-11ea-a6a4-bb031a9e1ba1', 'X-Knack-Application-Id':'5e13989941e72c0e039e117f'}
POST_HEADERS = GET_HEADERS.copy()
POST_HEADERS.update({'content-type': 'application/json'})
YESTERDAY_DATE = (date.today() - timedelta(days = 1)).strftime("%m/%d/%Y")
DATE_TO_RUN = '08/15/2022' #YESTERDAY_DATE

#sorting dict for output and other vals
sorting_dict = {
    'LocalDuesCategory':{
        'extras_array':[False, True, False],
        'sort_array':['User','Action','KnackFieldGUID','LocalDuesCategoryID','LocalDuesCategoryName','StatePerCapitaID','NationalPerCapitaID','LocalDuesAmount','LocalDuesPercentage','CreatedAt','UpdatedAt','DeletedAt','UpdateIndividualAffiliateToID','UpdateIndividualAffiliateToName','UpdateIndividualAffiliateToKnackGUID','AffiliateID','ProcessedInAFTDB']
    },
    'Employer':{
        'extras_array':[False, False, True],
        'sort_array':['User','Action','KnackFieldGUID','EmployerID','EmployerName','EmployerTypeID','ParentEmployerID','ParentEmployerName','ParentEmployerKnackGUID','Acronym','EmployerCode','Area','ChapterID','HasPrivateSector','IsStructural','IsUnknown','CreatedAt','UpdatedAt','DeletedAt','AffiliateID','ProcessedInAFTDB']
    },
    'LocalAgreement':{
        'extras_array':[True, False, False],
        'sort_array':['User','Action','KnackFieldGUID','LocalAgreementID','LocalAgreementName','EmployerID','EmployerName','EmployerKnackGUID','LocalAgreementTypeID','IsStructural','IsUnknown','CreatedAt','UpdatedAt','DeletedAt','AffiliateID','ProcessedInAFTDB']
    },
    'Unit':{
        'extras_array':[True, False, False],
        'sort_array':['User','Action','KnackFieldGUID','UnitID','UnitName','UnitTypeID','DivisionID','LocalAgreementID','LocalAgreementName','LocalAgreementKnackGUID','IsStructural','IsUnknown','CreatedAt','UpdatedAt','DeletedAt','AffiliateID','ProcessedInAFTDB']
    },
    'LocalJobClass':{
        'extras_array':[True, True, False],
        'sort_array':['User','Action','KnackFieldGUID','LocalJobClassID','LocalJobClassName','NationaJobClassID','UnitID','UnitName','UnitKnackGUID','IsStructural','IsUnknown','CreatedAt','UpdatedAt','DeletedAt','UpdateIndividualEmployerToID','UpdateIndividualEmployerToName','UpdateIndividualEmployerToKnackGUID','AffiliateID','ProcessedInAFTDB']
    },
    'JobTitle':{
        'extras_array':[True, True, False],
        'sort_array':['User','Action','KnackFieldGUID','JobTitleID','JobTitleName','LocalJobClassID','LocalJobClassName','LocalJobClassKnackGUID','IsStructural','IsUnknown','CreatedAt','UpdatedAt','DeletedAt','UpdateIndividualEmployerToID','UpdateIndividualEmployerToName','UpdateIndividualEmployerToKnackGUID','AffiliateID','ProcessedInAFTDB']
    },
    'WorkLocation':{
        'extras_array':[True, True, True],
        'sort_array':['User','Action','KnackFieldGUID','WorkLocationID','WorkLocationName','WorkLocationTypeID','NationalInstitutionTypeID','ParentWorkLocationID','ParentWorkLocationName','ParentWorkLocationKnackGUID','EmployerID','EmployerName','EmployerKnackGUID','WorkLocationCode','WorkLocationArea','CreatedAt','UpdatedAt','DeletedAt','UpdateIndividualEmployerToID','UpdateIndividualEmployerToName','UpdateIndividualEmployerToKnackGUID','AffiliateID','ProcessedInAFTDB']
    },
    'WorkStructure':{
        'extras_array':[True, True, True],
        'sort_array':['User','Action','KnackFieldGUID','WorkStructureID','WorkStructureName','WorkStructureTypeID','ParentWorkStructureID','ParentWorkStructureName','ParentWorkStructureKnackGUID','EmployerID','EmployerName','EmployerKnackGUID','WorkStructureCode','CreatedAt','UpdatedAt','DeletedAt','UpdateIndividualEmployerToID','UpdateIndividualEmployerToName','UpdateIndividualEmployerToKnackGUID','AffiliateID','ProcessedInAFTDB']
    }
}


# Knack entity mappings for readability
knackmappingdict = {
    'Affiliate': {'objectid':'object_25', 'ID':'field_126'},
    'Chapter': {'objectid':'object_26', 'ID':'field_148'},
    'Division': {'objectid':'object_17', 'ID':'field_108'},
    'EmployerType': {'objectid':'object_18', 'ID':'field_254'},
    'LocalAgreementType': {'objectid':'object_35', 'ID':'field_282'},
    'NationalInstitutionType': {'objectid':'object_38', 'ID':'field_377'},
    'NationaJobClass': {'objectid':'object_20', 'ID':'field_329'},
    'NationalPerCapita': {'objectid':'object_40', 'ID':'field_425'},
    'StatePerCapita': {'objectid':'object_39','ID':'field_438'},
    'UnitType': {'objectid':'object_36','ID':'field_309'},
    'User': {'objectid':'object_4','ID':'field_842'},
    'WorkLocationType': {'objectid':'object_19','ID':'field_366'},
    'WorkStructureType': {'objectid':'object_30','ID':'field_401'},
    'LocalDuesCategory':
    {
        'objectid':'object_37', 'ID':'field_451', 'GUID':'field_870',
        'connections':
        {
            'Entity-Affiliate':'field_448_raw',
            'Entity-StatePerCapita':'field_449_raw',
            'Entity-NationalPerCapita':'field_450_raw',
            'Entity-User':'field_470_raw'
        },
        'fields':
        {
            'LocalDuesCategoryName':'field_452',
            'LocalDuesAmount':'field_453',
            'LocalDuesPercentage':'field_454',
            'CreatedAt':'field_459',
            'mdate':'field_469',
            'DeletedAt':'field_462'
        },
        'move_to_fields':
        {
            'MoveTo':'field_579_raw'
        }
    },
    'Employer':
    {
        'objectid':'object_16', 'ID':'field_263', 'GUID':'field_854',
        'connections':
        {
            'Entity-Affiliate':'field_588_raw',
            'Entity-Chapter':'field_262_raw',
            'Entity-EmployerType':'field_261_raw',
            'Entity-User':'field_593_raw'
        },
        'fields':
        {
            'EmployerName':'field_265',
            'Acronym':'field_266',
            'EmployerCode':'field_267',
            'Area':'field_269',
            'HasPrivateSector':'field_268',
            'IsStructural':'field_271',
            'IsUnknown':'field_272',
            'CreatedAt':'field_274',
            'mdate':'field_592',
            'DeletedAt':'field_277'
        },
        'parent_fields':
        {
            'Entity-ParentEmployer':'field_278_raw'
        }
    },
    'LocalAgreement':
    {
        'objectid':'object_23', 'ID':'field_291', 'GUID':'field_856',
        'connections':
        {
            'Entity-Affiliate':'field_589_raw',
            'Entity-LocalAgreementType':'field_290_raw',
            'Entity-User':'field_724_raw'            
        },
        'fields':
        {
            'LocalAgreementName':'field_292',
            'IsStructural':'field_300',
            'IsUnknown':'field_301',
            'CreatedAt':'field_303',
            'mdate':'field_725',
            'DeletedAt':'field_306'
        },
        'mapped_fields':
        {
            'Entity-Employer':'field_289_raw'
        }
    },
    'Unit':
    {
        'objectid':'object_22', 'ID':'field_319', 'GUID':'field_858',
        'connections':
        {
            'Entity-Affiliate':'field_590_raw',
            'Entity-Division':'field_318_raw',
            'Entity-UnitType':'field_316_raw',
            'Entity-User':'field_722_raw'
            
        },
        'fields':
        {
            'UnitName':'field_321',
            'IsStructural':'field_322',
            'IsUnknown':'field_323',
            'CreatedAt':'field_325',
            'mdate':'field_723',
            'DeletedAt':'field_328'
        },
        'mapped_fields':
        {
            'Entity-LocalAgreement':'field_317_raw'
        }
    },
    'LocalJobClass':
    {
        'objectid':'object_21', 'ID':'field_343', 'GUID':'field_860',
        'connections':
        {
            'Entity-Affiliate':'field_591_raw',
            'Entity-NationaJobClass':'field_341_raw',
            'Entity-User':'field_716_raw'
        },
        'fields':
        {
            'LocalJobClassName':'field_345',
            'IsStructural':'field_347',
            'IsUnknown':'field_348',
            'CreatedAt':'field_350',
            'mdate':'field_717',
            'DeletedAt':'field_353'
        },
        'mapped_fields':
        {
            'Entity-Unit':'field_342_raw'
        },
        'move_to_fields':
        {
            'MoveTo':'field_650_raw'
        }
    },
    'JobTitle':
    {
        'objectid':'object_24', 'ID':'field_355', 'GUID':'field_862',
        'connections':
        {
            'Entity-Affiliate':'field_597_raw',
            'Entity-User':'field_601_raw'
        },
        'fields':
        {
            'JobTitleName':'field_356',
            'IsStructural':'field_358',
            'IsUnknown':'field_359',
            'CreatedAt':'field_361',
            'mdate':'field_602',
            'DeletedAt':'field_364'
        },
        'mapped_fields':
        {
            'Entity-LocalJobClass':'field_354_raw'
        },
        'move_to_fields':
        {
            'MoveTo':'field_651_raw'
        }
    },
    'WorkLocation':
    {
        'objectid':'object_28', 'ID':'field_389', 'GUID':'field_864',
        'connections':
        {
            'Entity-Affiliate':'field_598_raw',
            'Entity-NationalInstitutionType':'field_386_raw',
            'Entity-User':'field_603_raw',
            'Entity-WorkLocationType':'field_387_raw'
        },
        'fields':
        {
            'WorkLocationName':'field_391',
            'WorkLocationCode':'field_392',
            'WorkLocationArea':'field_393',
            'CreatedAt':'field_396',
            'mdate':'field_604',
            'DeletedAt':'field_399'
        },
        'mapped_fields':
        {
            'Entity-Employer':'field_385_raw'
        },
        'move_to_fields':
        {
            'MoveTo':'field_654_raw'
        },
        'parent_fields':
        {
            'Entity-ParentWorkLocation':'field_388_raw'
        }
    },
    'WorkStructure':
    {
        'objectid':'object_29', 'ID':'field_412', 'GUID':'field_866',
        'connections':
        {
            'Entity-Affiliate':'field_599_raw',
            'Entity-User':'field_605_raw',
            'Entity-WorkStructureType':'field_410_raw'
        },
        'fields':
        {
            'WorkStructureName':'field_414',
            'WorkStructureCode':'field_415',
            'CreatedAt':'field_417',
            'mdate':'field_606',
            'DeletedAt':'field_420'
        },
        'mapped_fields':
        {
            'Entity-Employer':'field_409_raw'
        },
        'move_to_fields':
        {
            'MoveTo':'field_656_raw'
        },
        'parent_fields':
        {
            'Entity-ParentWorkStructure':'field_411_raw'
        }
    }
}

# GET and format json from requestURL
def getJson(request_url):
    r = requests.get(url = request_url, headers = GET_HEADERS)
    return json.dumps(r.json(), indent=4)

# Find object_id of a given knack object
def find_object_id(knack_object):
    return knackmappingdict[knack_object]['objectid']


# Find field_id of a given knack object and field_name
def find_field_id(knack_object, field_name):
    return knackmappingdict[knack_object]['fields'][field_name]


def find_records_updated_at_date(knack_object, date):
    #Convert to IDs
    knack_object_id = find_object_id(knack_object)
    field_to_match_id = find_field_id(knack_object, "mdate")
    
    #Get Id
    match_filter = {'match':'and', 'rules':[{'field':field_to_match_id, 'operator':'is', 'value': date}]}
    filter_for_url = urllib.parse.quote(json.dumps(match_filter))
    request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records?filters=" + filter_for_url
    
    r = requests.get(url = request_url, headers = GET_HEADERS)
    #print(json.dumps(r.json(), indent=4))
    res_json_dict = json.loads(json.dumps(r.json()))
    if res_json_dict["total_records"] == 0:
        return ''
    else:
        return res_json_dict["records"]

    
def find_connection_payload(name, lookup):
    output = {}
    knack_object_id = find_object_id(name)
    if lookup:
        request_url = "https://api.knack.aft.org/v1/objects/" + knack_object_id + "/records/" + lookup
        r = requests.get(url = request_url, headers = GET_HEADERS)
        output = json.loads(json.dumps(r.json()))
        return output
    else:
        return ''
    

def getFields(record, object_name):
    obj = knackmappingdict[object_name]
    output = {}
    
    conn_values = obj['fields']
    conn_values.update({(object_name+'ID'):obj['ID'], 'KnackFieldGUID':obj['GUID']})
    
    for k,v in conn_values.items():
        if record[v]:
            data = record[v]
        else:
            data = ''
        output.update({k:data})
        
    output['UpdatedAt'] = output.pop('mdate')
    
    if output[(object_name+'ID')]:
        if output['DeletedAt']:
            output.update({'Action':'Delete'})
        else:
            output.update({'Action':'Update'})
    else:
        if output['DeletedAt']:
            output = {}
        else:
            output.update({'Action':'Insert'})
    if output:
        output.update({'ProcessedInAFTDB':'0'})
    #print(yaml.dump(dict_out, default_flow_style=False))
    return output


def getConnections(record, object_name):
    obj = knackmappingdict[object_name]
    output = {}
    
    conn_values = obj['connections']
    
    for k,v in conn_values.items():
        fieldval = v.replace('_raw', '')
        res = ''
        knack_object = k.replace('Entity-', '')
        if record[fieldval]:
            data = record[v][0]['id']
            field_id = knackmappingdict[knack_object]['ID']
            res = find_connection_payload(knack_object, data)[field_id]
        output.update({(knack_object+'ID'):res})
        
    output['User'] = output.pop('UserID')
    return output
    

def getMappedFields(record, object_name):
    conn_values = knackmappingdict[object_name]['mapped_fields']
    for k,v in conn_values.items():
        entity = k.replace('Entity-', '')
        output = {entity+'ID':'',entity+'Name':'', entity+'KnackGUID':''}
        fieldval = v.replace('_raw', '')
        if record[v]:
            data = record[v][0]['id']
                
            field_id = knackmappingdict[entity]['ID']
            field_guid = knackmappingdict[entity]['GUID']
            field_name = knackmappingdict[entity]['fields'][(entity+'Name')]
                
            res = find_connection_payload(entity, data)
            out_id = res[field_id]
            out_guid = res[field_guid]
            out_name = res[field_name]
                
            output.update({entity+'ID':out_id,entity+'Name':out_name, entity+'KnackGUID':out_guid})
    
    #print(yaml.dump(dict_out, default_flow_style=False))
    return output


def getParent(record, object_name):
    output = {'Parent'+object_name+'ID':'','Parent'+object_name+'Name':'', 'Parent'+object_name+'KnackGUID':''}
    conn_values = knackmappingdict[object_name]['parent_fields']
    for k,v in conn_values.items():
        if 'Parent' in k:
            fieldval = v.replace('_raw', '')
            if record[fieldval]:
                data = record[v][0]['id']
                
                field_id = knackmappingdict[object_name]['ID']
                field_guid = knackmappingdict[object_name]['GUID']
                field_name = knackmappingdict[object_name]['fields'][(object_name+'Name')]
                
                res = find_connection_payload(object_name, data)
                out_id = res[field_id]
                out_guid = res[field_guid]
                out_name = res[field_name]
                
                output.update({'Parent'+object_name+'ID':out_id,'Parent'+object_name+'Name':out_name, 'Parent'+object_name+'KnackGUID':out_guid})
    
    #print(yaml.dump(dict_out, default_flow_style=False))
    return output


def getMoveTo(record, object_name):
    if object_name == 'LocalDuesCategory':
        phelper = 'Affiliate'
    else:
        phelper = 'Employer'
        
    output = {'UpdateIndividual'+phelper+'ToID':'','UpdateIndividual'+phelper+'ToName':'', 'UpdateIndividual'+phelper+'ToKnackGUID':''}
    conn_values = knackmappingdict[object_name]['move_to_fields']
    for k,v in conn_values.items():
        if 'MoveTo' in k:
            fieldval = v.replace('_raw', '')
            if record[fieldval]:
                data = record[v][0]['id']
                
                field_id = knackmappingdict[object_name]['ID']
                field_guid = knackmappingdict[object_name]['GUID']
                field_name = knackmappingdict[object_name]['fields'][(object_name+'Name')]
                
                res = find_connection_payload(object_name, data)
                out_id = res[field_id]
                out_guid = res[field_guid]
                out_name = res[field_name]
                
                output.update({'UpdateIndividual'+phelper+'ToID':out_id,'UpdateIndividual'+phelper+'ToName':out_name, 'UpdateIndividual'+phelper+'ToKnackGUID':out_guid})
    
    #print(yaml.dump(dict_out, default_flow_style=False))
    return output


def payload_runner(entity_name, mapped, moveTo, parent):
    payload = find_records_updated_at_date(entity_name, DATE_TO_RUN)
    out = []
    #print(yaml.dump(payload, default_flow_style=False))
    for record in payload:
        fields = getFields(record, entity_name)
        connections = getConnections(record, entity_name)
        output1 = {**fields, **connections}

        if (mapped):
            
            # MAPPED FIELDS
            output2 = getMappedFields(record, entity_name)
            output1 = {**output1, **output2}
        if (moveTo):
            
            # MOVE TO FIELDS
            output2 = getMoveTo(record, entity_name)
            output1 = {**output1, **output2}
        if (parent):
            
            # PARENT FIELDS
            output2 = getParent(record, entity_name)
            output1 = {**output1, **output2}
        
        if fields:
            out.append(output1)
    return pd.DataFrame(out, dtype=str) 
            
def entity_runner(entity):
    print('Running Entity: '+entity+'...')

    extras = sorting_dict[entity]['extras_array']
    sort = sorting_dict[entity]['sort_array']

    output_df = payload_runner(entity, extras[0], extras[1], extras[2])
    rows = output_df.shape[0]
    columns = output_df.shape[1]

    if rows == 0 or columns == 0:
        print('No Changes found for '+entity+'!')
        print('Skipping write to civis...')
    else:
        print('Found '+str(rows)+' rows!')

        output_df = output_df[sort]
        entity = entity.lower()
        table_name = os.environ['table_schema'] + entity

        print('Fake writing to civis table: ' + table_name + '...')
        print(output_df)
        print('Done with debug output!')

def main():
    list_to_run =['LocalDuesCategory','Employer','LocalAgreement','Unit','LocalJobClass','JobTitle','WorkLocation','WorkStructure']
    for entity in list_to_run:
        print("================================")
        entity_runner(entity)
    print("================================")

main()

In [ ]:
# Knack entity mappings for readability
knackmappingdict = {
    'Affiliate': {'objectid':'object_25', 'ID':'field_126'},
    'Chapter': {'objectid':'object_26', 'ID':'field_148'},
    'Division': {'objectid':'object_17', 'ID':'field_108'},
    'EmployerType': {'objectid':'object_18', 'ID':'field_254'},
    'LocalAgreementType': {'objectid':'object_35', 'ID':'field_282'},
    'NationalInstitutionType': {'objectid':'object_38', 'ID':'field_377'},
    'NationaJobClass': {'objectid':'object_20', 'ID':'field_329'},
    'NationalPerCapita': {'objectid':'object_40', 'ID':'field_425'},
    'StatePerCapita': {'objectid':'object_39','ID':'field_438'},
    'UnitType': {'objectid':'object_36','ID':'field_309'},
    'User': {'objectid':'object_4','ID':'field_842'},
    'WorkLocationType': {'objectid':'object_19','ID':'field_366'},
    'WorkStructureType': {'objectid':'object_30','ID':'field_401'},
    'LocalDuesCategory':
    {
        'objectid':'object_37', 'ID':'field_451', 'GUID':'field_870',
        'connections':
        {
            'Entity-Affiliate':'field_448_raw',
            'Entity-StatePerCapita':'field_449_raw',
            'Entity-NationalPerCapita':'field_450_raw',
            'Entity-User':'field_470_raw'
        },
        'fields':
        {
            'LocalDuesCategoryName':'field_452',
            'LocalDuesAmount':'field_453',
            'LocalDuesPercentage':'field_454',
            'CreatedAt':'field_459',
            'mdate':'field_469',
            'DeletedAt':'field_462'
        },
        'move_to_fields':
        {
            'MoveTo':'field_579_raw'
        }
    },
    'Employer':
    {
        'objectid':'object_16', 'ID':'field_263', 'GUID':'field_854',
        'connections':
        {
            'Entity-Affiliate':'field_588_raw',
            'Entity-Chapter':'field_262_raw',
            'Entity-EmployerType':'field_261_raw',
            'Entity-User':'field_593_raw'
        },
        'fields':
        {
            'EmployerName':'field_265',
            'Acronym':'field_266',
            'EmployerCode':'field_267',
            'Area':'field_269',
            'HasPrivateSector':'field_268',
            'IsStructural':'field_271',
            'IsUnknown':'field_272',
            'CreatedAt':'field_274',
            'mdate':'field_592',
            'DeletedAt':'field_277'
        },
        'parent_fields':
        {
            'Entity-ParentEmployer':'field_278_raw'
        }
    },
    'LocalAgreement':
    {
        'objectid':'object_23', 'ID':'field_291', 'GUID':'field_856',
        'connections':
        {
            'Entity-Affiliate':'field_589_raw',
            'Entity-LocalAgreementType':'field_290_raw',
            'Entity-User':'field_724_raw'            
        },
        'fields':
        {
            'LocalAgreementName':'field_292',
            'IsStructural':'field_300',
            'IsUnknown':'field_301',
            'CreatedAt':'field_303',
            'mdate':'field_725',
            'DeletedAt':'field_306'
        },
        'mapped_fields':
        {
            'Entity-Employer':'field_289_raw'
        }
    },
    'Unit':
    {
        'objectid':'object_22', 'ID':'field_319', 'GUID':'field_858',
        'connections':
        {
            'Entity-Affiliate':'field_590_raw',
            'Entity-Division':'field_318_raw',
            'Entity-UnitType':'field_316_raw',
            'Entity-User':'field_722_raw'
            
        },
        'fields':
        {
            'UnitName':'field_321',
            'IsStructural':'field_322',
            'IsUnknown':'field_323',
            'CreatedAt':'field_325',
            'mdate':'field_723',
            'DeletedAt':'field_328'
        },
        'mapped_fields':
        {
            'Entity-LocalAgreement':'field_317_raw'
        }
    },
    'LocalJobClass':
    {
        'objectid':'object_21', 'ID':'field_343', 'GUID':'field_860',
        'connections':
        {
            'Entity-Affiliate':'field_591_raw',
            'Entity-NationaJobClass':'field_341_raw',
            'Entity-User':'field_716_raw'
        },
        'fields':
        {
            'LocalJobClassName':'field_345',
            'IsStructural':'field_347',
            'IsUnknown':'field_348',
            'CreatedAt':'field_350',
            'mdate':'field_717',
            'DeletedAt':'field_353'
        },
        'mapped_fields':
        {
            'Entity-Unit':'field_342_raw'
        },
        'move_to_fields':
        {
            'MoveTo':'field_650_raw'
        }
    },
    'JobTitle':
    {
        'objectid':'object_24', 'ID':'field_355', 'GUID':'field_862',
        'connections':
        {
            'Entity-Affiliate':'field_597_raw',
            'Entity-User':'field_601_raw'
        },
        'fields':
        {
            'JobTitleName':'field_356',
            'IsStructural':'field_358',
            'IsUnknown':'field_359',
            'CreatedAt':'field_361',
            'mdate':'field_602',
            'DeletedAt':'field_364'
        },
        'mapped_fields':
        {
            'Entity-LocalJobClass':'field_354_raw'
        },
        'move_to_fields':
        {
            'MoveTo':'field_651_raw'
        }
    },
    'WorkLocation':
    {
        'objectid':'object_28', 'ID':'field_389', 'GUID':'field_864',
        'connections':
        {
            'Entity-Affiliate':'field_598_raw',
            'Entity-NationalInstitutionType':'field_386_raw',
            'Entity-User':'field_603_raw',
            'Entity-WorkLocationType':'field_387_raw'
        },
        'fields':
        {
            'WorkLocationName':'field_391',
            'WorkLocationCode':'field_392',
            'WorkLocationArea':'field_393',
            'CreatedAt':'field_396',
            'mdate':'field_604',
            'DeletedAt':'field_399'
        },
        'mapped_fields':
        {
            'Entity-Employer':'field_385_raw'
        },
        'move_to_fields':
        {
            'MoveTo':'field_654_raw'
        },
        'parent_fields':
        {
            'Entity-ParentWorkLocation':'field_388_raw'
        }
    },
    'WorkStructure':
    {
        'objectid':'object_29', 'ID':'field_412', 'GUID':'field_866',
        'connections':
        {
            'Entity-Affiliate':'field_599_raw',
            'Entity-User':'field_605_raw',
            'Entity-WorkStructureType':'field_410_raw'
        },
        'fields':
        {
            'WorkStructureName':'field_414',
            'WorkStructureCode':'field_415',
            'CreatedAt':'field_417',
            'mdate':'field_606',
            'DeletedAt':'field_420'
        },
        'mapped_fields':
        {
            'Entity-Employer':'field_409_raw'
        },
        'move_to_fields':
        {
            'MoveTo':'field_656_raw'
        },
        'parent_fields':
        {
            'Entity-ParentWorkStructure':'field_411_raw'
        }
    }
}


def getMappedFields(record, object_name):
    conn_values = knackmappingdict[object_name]['mapped_fields']
    for k,v in conn_values.items():
        entity = k.replace('Entity-', '')
        output = {entity+'ID':'',entity+'Name':'', entity+'KnackGUID':''}
        fieldval = v.replace('_raw', '')
        if record[v]:
            data = record[v][0]['id']
                
            field_id = knackmappingdict[entity]['ID']
            field_guid = knackmappingdict[entity]['GUID']
            field_name = knackmappingdict[entity]['fields'][(entity+'Name')]
                
            res = find_connection_payload(entity, data)
            out_id = res[field_id]
            out_guid = res[field_guid]
            out_name = res[field_name]
                
            output.update({entity+'ID':out_id,entity+'Name':out_name, entity+'KnackGUID':out_guid})
    
    #print(yaml.dump(dict_out, default_flow_style=False))
    return output


def getParent(record, object_name):
    output = {'Parent'+object_name+'ID':'','Parent'+object_name+'Name':'', 'Parent'+object_name+'KnackGUID':''}
    conn_values = knackmappingdict[object_name]['parent_fields']
    for k,v in conn_values.items():
        if 'Parent' in k:
            fieldval = v.replace('_raw', '')
            if record[fieldval]:
                data = record[v][0]['id']
                
                field_id = knackmappingdict[object_name]['ID']
                field_guid = knackmappingdict[object_name]['GUID']
                field_name = knackmappingdict[object_name]['fields'][(object_name+'Name')]
                
                res = find_connection_payload(object_name, data)
                out_id = res[field_id]
                out_guid = res[field_guid]
                out_name = res[field_name]
                
                output.update({'Parent'+object_name+'ID':out_id,'Parent'+object_name+'Name':out_name, 'Parent'+object_name+'KnackGUID':out_guid})
    
    #print(yaml.dump(dict_out, default_flow_style=False))
    return output


def getMoveTo(record, object_name):
    if object_name == 'LocalDuesCategory':
        phelper = 'Affiliate'
    else:
        phelper = 'Employer'
        
    output = {'UpdateIndividual'+phelper+'ToID':'','UpdateIndividual'+phelper+'ToName':'', 'UpdateIndividual'+phelper+'ToKnackGUID':''}
    conn_values = knackmappingdict[object_name]['move_to_fields']
    for k,v in conn_values.items():
        if 'MoveTo' in k:
            fieldval = v.replace('_raw', '')
            if record[fieldval]:
                data = record[v][0]['id']
                
                field_id = knackmappingdict[object_name]['ID']
                field_guid = knackmappingdict[object_name]['GUID']
                field_name = knackmappingdict[object_name]['fields'][(object_name+'Name')]
                
                res = find_connection_payload(object_name, data)
                out_id = res[field_id]
                out_guid = res[field_guid]
                out_name = res[field_name]
                
                output.update({'UpdateIndividual'+phelper+'ToID':out_id,'UpdateIndividual'+phelper+'ToName':out_name, 'UpdateIndividual'+phelper+'ToKnackGUID':out_guid})
    
    #print(yaml.dump(dict_out, default_flow_style=False))
    return output


def payload_runner(entity_name, mapped, moveTo, parent):
    payload = find_records_updated_at_date(entity_name, DATE_TO_RUN)
    out = []
    #print(yaml.dump(payload, default_flow_style=False))
    for record in payload:
        fields = getFields(record, entity_name)
        connections = getConnections(record, entity_name)
        output1 = {**fields, **connections}

        if (mapped):
            
            # MAPPED FIELDS
            output2 = getMappedFields(record, entity_name)
            output1 = {**output1, **output2}
        if (moveTo):
            
            # MOVE TO FIELDS
            output2 = getMoveTo(record, entity_name)
            output1 = {**output1, **output2}
        if (parent):
            
            # PARENT FIELDS
            output2 = getParent(record, entity_name)
            output1 = {**output1, **output2}
        
        if fields:
            out.append(output1)
    return pd.DataFrame(out, dtype=str) 
            
def entity_runner(entity):
    print('Running Entity: '+entity+'...')

    extras = sorting_dict[entity]['extras_array']
    sort = sorting_dict[entity]['sort_array']

    output_df = payload_runner(entity, extras[0], extras[1], extras[2])
    rows = output_df.shape[0]
    columns = output_df.shape[1]

    if rows == 0 or columns == 0:
        print('No Changes found for '+entity+'!')
        print('Skipping write to civis...')
    else:
        print('Found '+str(rows)+' rows!')

        output_df = output_df[sort]
        entity = entity.lower()
        table_name = os.environ['table_schema'] + entity

        print('Fake writing to civis table: ' + table_name + '...')
        print(output_df)
        print('Done with debug output!')

def main():
    list_to_run =['LocalDuesCategory','Employer','LocalAgreement','Unit','LocalJobClass','JobTitle','WorkLocation','WorkStructure']
    for entity in list_to_run:
        print("================================")
        entity_runner(entity)
    print("================================")
